# RGB fits を作成します。
- [n2-tools](https://github.com/nanten2/n2-tools/tree/master/doc) の使い方はこちら

In [1]:
import pathlib
import numpy
import astropy.io.fits
import PIL
import n2

[10:50:49,906] INFO: python 3.6.9
[10:50:49,908] INFO: n2-tools 0.0.26


# 関数の定義
- 今回は内容が膨大なので、実験コードは下に載せることにします。
- get_fits_paths: 銀経を渡すと、その銀経を含む FITS の Path を list で返します。(MIPSGAL と GLIMPSE で分けています)<br>銀経は 3 の倍数推奨
- _nparr_sum, _new_header: make_rgb_fits で使用します。
- make_rgb_fits: get_fits_paths で取得した path をつなげて RGB FITS にして保存します。

In [2]:
def get_fits_paths(l):
    l = (lambda l: (l//3+1)*3 if l%3 > 1.5 else (l//3)*3)(l)
    path_mips = pathlib.Path('../data/raw/mipsgal/').expanduser().resolve()
    path_glim = pathlib.Path('../data/raw/glimpse/').expanduser().resolve() 
    
    paths_mips = [
        path_mips/'mips24/MG{}0n005_024.fits'.format('0'*(3-len(str(l-1)))+str(l-1)),
        path_mips/'mips24/MG{}0p005_024.fits'.format('0'*(3-len(str(l-1)))+str(l-1)),
        path_mips/'mips24/MG{}0n005_024.fits'.format('0'*(3-len(str(l)))+str(l)),
        path_mips/'mips24/MG{}0p005_024.fits'.format('0'*(3-len(str(l)))+str(l)),
        path_mips/'mips24/MG{}0n005_024.fits'.format('0'*(3-len(str(l+1)))+str(l+1)),
        path_mips/'mips24/MG{}0p005_024.fits'.format('0'*(3-len(str(l+1)))+str(l+1)),
    ]
    
    paths_glim = [
        path_glim/'irac1/GLM_{}00+0000_mosaic_I1.fits'.format('0'*(3-len(str(l)))+str(l)),
        path_glim/'irac4/GLM_{}00+0000_mosaic_I4.fits'.format('0'*(3-len(str(l)))+str(l)),
    ]
    
    if l == 0:
        paths_mips[0] = path_mips/'mips24/MG3590n005_024.fits'
        paths_mips[1] = path_mips/'mips24/MG3590p005_024.fits'
    else:
        pass
    
    return paths_mips, paths_glim

In [3]:
def _nparr_sum(nparr1, nparr2):
    r1 = numpy.nan_to_num(nparr1)
    r2 = numpy.nan_to_num(nparr2)
    r1_bool = numpy.where(r1==0, False, True)
    r2_bool = numpy.where(r2==0, False, True)
    r_bool = numpy.logical_and(r1_bool, r2_bool)
    r = numpy.where(r_bool==True, (r1+r2)/2, r1+r2)
    return r

def _new_header(header):
    header.pop('HISTORY*')
    header.pop('N2HASH')
    return header

In [4]:
def make_rgb_fits(paths_mips, paths_glim, save_path='../data/interim/gal'):
    save_path = pathlib.Path(save_path).expanduser().resolve()
    save_path = save_path/('spitzer_' + paths_glim[1].name[4:14] + '_rgb')
    save_path.mkdir(parents=True, exist_ok=True)
    
    rs_hdu_raw = [n2.open_fits(i) for i in paths_mips]
    g_hdu_raw = n2.open_fits(paths_glim[1])
    b_hdu_raw = n2.open_fits(paths_glim[0])
    
    header = g_hdu_raw.hdu.header.copy()
    rs_hdu = [i.regrid(header) for i in rs_hdu_raw]
    #g_hdu = g_hdu_raw.regrid(header)
    g_hdu = g_hdu_raw
    b_hdu = b_hdu_raw.regrid(header)
    
    r1 = _nparr_sum(rs_hdu[0].data, rs_hdu[1].data)
    r2 = _nparr_sum(rs_hdu[2].data, rs_hdu[3].data)
    r3 = _nparr_sum(rs_hdu[4].data, rs_hdu[5].data)
    r4 = _nparr_sum(r1, r2)
    r = _nparr_sum(r3, r4)    
    g = numpy.nan_to_num(g_hdu.data)
    b = numpy.nan_to_num(b_hdu.data)
    
    r_hdu = astropy.io.fits.PrimaryHDU(r, _new_header(rs_hdu[0].header))
    g_hdu = astropy.io.fits.PrimaryHDU(g, _new_header(g_hdu.header))
    b_hdu = astropy.io.fits.PrimaryHDU(b, _new_header(b_hdu.header))    
    r_hdu_list = astropy.io.fits.HDUList([r_hdu])
    g_hdu_list = astropy.io.fits.HDUList([g_hdu])
    b_hdu_list = astropy.io.fits.HDUList([b_hdu])
    r_hdu_list.writeto(save_path/'r.fits', overwrite=True)
    g_hdu_list.writeto(save_path/'g.fits', overwrite=True)
    b_hdu_list.writeto(save_path/'b.fits', overwrite=True)
    
    return

# 使用例: 一気に全部作っちゃう

In [5]:
l = [i for i in range(0, 66, 3)]
for i in l:
    make_rgb_fits(*get_fits_paths(i))
    
l = [i for i in range(297, 360, 3)]
for i in l:
    make_rgb_fits(*get_fits_paths(i))

In [6]:
rgb = astropy.io.fits.open('../data/interim/gal/spitzer_gal_01200+0000_rgb.fits')[0]
r = numpy.where(rgb.data[0]>255, 255, rgb.data[0])
r = numpy.where(r<0, 0, r)
g = numpy.where(rgb.data[1]>255, 255, rgb.data[1])
g = numpy.where(g<0, 0, g)
b = numpy.where(rgb.data[2]>255, 255, rgb.data[2])
b = numpy.where(b<0, 0, b)
rgb = numpy.stack([r, g, b], 2)
im = PIL.Image.fromarray(numpy.uint8(rgb))
#im

# これより下、上の関数を作るために実験など繰り返した痕跡

### 波長毎の座標範囲(l)
- mips24: 75 - 105
- irac1 : 75 - 105
- irac4 : 75 - 105

In [7]:
path_mips = pathlib.Path('~/data/spitzer/mipsgal/').expanduser().resolve()
path_glim = pathlib.Path('~/data/spitzer/glimpse/').expanduser().resolve()
files_mips = [
    'mips24/MG0080n005_024.fits',
    'mips24/MG0080p005_024.fits',
    'mips24/MG0090n005_024.fits',
    'mips24/MG0090p005_024.fits',
    'mips24/MG0100n005_024.fits',
    'mips24/MG0100p005_024.fits',
]
files_glim = [
    'irac1/GLM_00900+0000_mosaic_I1.fits',
    'irac4/GLM_00900+0000_mosaic_I4.fits',
]
r_1 = n2.open_fits(path_mips/files_mips[0])
r_2 = n2.open_fits(path_mips/files_mips[1])
r_3 = n2.open_fits(path_mips/files_mips[2])
r_4 = n2.open_fits(path_mips/files_mips[3])
r_5 = n2.open_fits(path_mips/files_mips[4])
r_6 = n2.open_fits(path_mips/files_mips[5])

g = n2.open_fits(path_glim/files_glim[1])
b = n2.open_fits(path_glim/files_glim[0])

In [8]:
header = g.hdu.header
data = g.hdu.data
#header
#data

In [9]:
header1 = g.header.copy()

r_1_re = r_1.regrid(header1)
r_2_re = r_2.regrid(header1)
r_3_re = r_3.regrid(header1)
r_4_re = r_4.regrid(header1)
r_5_re = r_5.regrid(header1)
r_6_re = r_6.regrid(header1)

g_re = g.regrid(header1)
b_re = b.regrid(header1)

In [10]:
PIL.Image.fromarray(
    numpy.uint8(
        numpy.nan_to_num(r_1_re.data) + \
        numpy.nan_to_num(r_2_re.data) + \
        numpy.nan_to_num(r_3_re.data) + \
        numpy.nan_to_num(r_4_re.data) + \
        numpy.nan_to_num(r_5_re.data) + \
        numpy.nan_to_num(r_6_re.data)
    )
)

In [11]:
a = numpy.nan_to_num(r_1_re.data)
a = numpy.where(a==0, False, True)
b = numpy.nan_to_num(r_2_re.data)
b = numpy.where(b==0, False, True)
c = numpy.logical_and(a, b)
#c = numpy.where(c==False, 0, 255)
#PIL.Image.fromarray(numpy.uint8(a))
#PIL.Image.fromarray(numpy.uint8(b))
#PIL.Image.fromarray(numpy.uint8(c))

test1 = numpy.nan_to_num(r_1_re.data) + numpy.nan_to_num(r_2_re.data)
test1 = numpy.where(c==True, test1/2, test1)
#PIL.Image.fromarray(numpy.uint8(test1))

In [12]:
a = numpy.nan_to_num(r_3_re.data)
a = numpy.where(a==0, False, True)
b = numpy.nan_to_num(r_4_re.data)
b = numpy.where(b==0, False, True)
c = numpy.logical_and(a, b)
#c = numpy.where(c==False, 0, 255)
#PIL.Image.fromarray(numpy.uint8(a))
#PIL.Image.fromarray(numpy.uint8(b))
#PIL.Image.fromarray(numpy.uint8(c))

test2 = numpy.nan_to_num(r_3_re.data) + numpy.nan_to_num(r_4_re.data)
test2 = numpy.where(c==True, test2/2, test2)
#PIL.Image.fromarray(numpy.uint8(test2))

In [13]:
a = numpy.nan_to_num(r_5_re.data)
a = numpy.where(a==0, False, True)
b = numpy.nan_to_num(r_6_re.data)
b = numpy.where(b==0, False, True)
c = numpy.logical_and(a, b)
#c = numpy.where(c==False, 0, 255)
#PIL.Image.fromarray(numpy.uint8(a))
#PIL.Image.fromarray(numpy.uint8(b))
#PIL.Image.fromarray(numpy.uint8(c))

test3 = numpy.nan_to_num(r_5_re.data) + numpy.nan_to_num(r_6_re.data)
test3 = numpy.where(c==True, test3/2, test3)
#PIL.Image.fromarray(numpy.uint8(test3))

In [14]:
a = numpy.where(test1==0, False, True)
b = numpy.where(test2==0, False, True)
c = numpy.where(test3==0, False, True)
d = numpy.logical_and(a, b)
e = numpy.logical_and(b, c)
f = numpy.logical_or(d, e)
#f = numpy.where(f==False, 0, 255)
#PIL.Image.fromarray(numpy.uint8(a))
#PIL.Image.fromarray(numpy.uint8(b))
#PIL.Image.fromarray(numpy.uint8(c))
#PIL.Image.fromarray(numpy.uint8(d))
#PIL.Image.fromarray(numpy.uint8(e))
#PIL.Image.fromarray(numpy.uint8(f))

test4 = test1 + test2 + test3
test4 = numpy.where(f==True, test4/2, test4)
#PIL.Image.fromarray(numpy.uint8(test4))

In [15]:
r_re = r_1_re
r_re.data = test4
#PIL.Image.fromarray(numpy.uint8(r_re.data))

In [16]:
g_re.data = numpy.nan_to_num(g_re.data)
b_re.data = numpy.nan_to_num(b_re.data)

In [17]:
img = n2.jpy_rgbimage(r_re.data, g_re.data, b_re.data, nanval='min', qlook_size=500)
img